# Take Control of GPT : Use the API. Don't let the API use you

### My first openai API Project!!!

Here, I use the openai Python lib and some custom classes to handle processing prompts and responses. I store the results in a pandas dataframe, then format the code in a pandas 'style' object. I can use the dataframe and other objects (like the multiline string of prompts) to manage context for later prompts. Yes, context from prompts and answers. So much more to do with this. But this is a start.

## Setup

In [1]:
%%html
<style>
    /* Enable word wrap */
    .rendered_html table, .rendered_html th, .rendered_html tr, .rendered_html td {
      white-space: nowrap;
      overflow: hidden;
      text-overflow: ellipsis;
    }
</style>

### Imports

In [2]:
# ...
import openai
import os
import pprint
import sys
from pprint import pprint
import json
import pyarrow as pa
import pandas as pd

lib = os.path.abspath(os.path.join('..', '..', 'libs'))
sys.path.append(lib)

from MessageParser import MessageParser
from ResponseParser import ResponseParser
from PromptLogs import PromptLogs


### Config

In [3]:
# ...
# *******************************************************************************************************
# SET THE API KEY
# ================
# -- set the env variable OPENAI_API_KEY to your OpenAI API key       <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
openai.api_key = os.getenv("OPENAI_API_KEY")
# *******************************************************************************************************

# Setup up pandas properties
pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.expand_frame_repr', False)  # Prevent line break in DataFrame
pd.set_option('display.max_colwidth', None)  # Display the full content of cells



## Main code

Prep the system prompts

In [4]:
sys_prompt_list = []
sys_prompt_1 = """You are a software expert."""
sys_prompt_2 = """Use. "```python" style for code blocks."""
sys_prompt_3 = """Include the name for a file where I would store individual code for functions or classes."""

sys_prompt_list.append(sys_prompt_1)
sys_prompt_list.append(sys_prompt_2)
sys_prompt_list.append(sys_prompt_3)

sys_prompt_string = "".join(sys_prompt_list)

# we'll be storing the individual parser logs here 

In [5]:
logs = PromptLogs()

Generate the message and make the call

In [6]:
message_list = f"""
system: {sys_prompt_string}
user: Write python code for fibonacci sequence. also write a function to zero pad a string to 10 chars.
"""

parser = MessageParser(message_list)
messages = parser.parse_messages()

# Get the response
# I'll abstract this away in the future
response = openai.ChatCompletion.create(model='gpt-3.5-turbo', temperature=1.3, max_tokens=300, messages = messages)


Get a parser object, using a custom class.

In [7]:
parser = ResponseParser(response=response)

get a pandas dataframe of the logged prompts and responses

In [8]:
log_df = parser.get_df()
log_df

,id,created,model,usage,all_code,code_blocks,content
0,chatcmpl-73yLvjyfoIYg0n9OGyQibvV7HyHNx,1.681181e+09,gpt-3.5-turbo-0301,66.0,#&nbsp;fibonacci.py<br><br>def&nbsp;fibonacci(n:&nbsp;int)&nbsp;->&nbsp;int:<br>&nbsp;&nbsp;&nbsp;&nbsp;if&nbsp;n&nbsp;<=&nbsp;1:<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;return&nbsp;n<br>&nbsp;&nbsp;&nbsp;&nbsp;else:<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;return&nbsp;(fibonacci(n-1)&nbsp;+&nbsp;fibonacci(n-2))<br><br>def&nbsp;zero_pad(string:&nbsp;str)&nbsp;->&nbsp;str:<br>&nbsp;&nbsp;&nbsp;&nbsp;if&nbsp;len(string)&nbsp;>=&nbsp;10:<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;return&nbsp;string<br>&nbsp;&nbsp;&nbsp;&nbsp;else:<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;pad&nbsp;=&nbsp;'0'&nbsp;*&nbsp;(10&nbsp;-&nbsp;len(string))<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;return&nbsp;pad&nbsp;+&nbsp;string<br>,[\n# fibonacci.py\n\ndef fibonacci(n: int) -> int:\n if n <= 1:\n return n\n else:\n return (fibonacci(n-1) + fibonacci(n-2))\n\ndef zero_pad(string: str) -> str:\n if len(string) >= 10:\n return string\n else:\n pad = '0' * (10 - len(string))\n return pad + string\n],"```python\n# fibonacci.py\n\ndef fibonacci(n: int) -> int:\n if n <= 1:\n return n\n else:\n return (fibonacci(n-1) + fibonacci(n-2))\n\ndef zero_pad(string: str) -> str:\n if len(string) >= 10:\n return string\n else:\n pad = '0' * (10 - len(string))\n return pad + string\n```\nIn the above code, we have defined two functions: \n\n- `fibonacci(n: int) -> int`: returns the ""n""th Fibonacci number.\n- `zero_pad(string: str) -> str`: Takes a string as input and adds extra 0s to it to make it 10 characters long. If the string is already 10 or more characters, it is returned as-is. \n\nWe have saved these functions in the file `fibonacci.py`."


In [9]:
# this is a PromptLogs object
logs.append(parser)

In [10]:
logs.get_df()

,created,model,usage,all_code,code_blocks,content
id,,,,,,
chatcmpl-73yLvjyfoIYg0n9OGyQibvV7HyHNx,1.681181e+09,gpt-3.5-turbo-0301,66.0,#&nbsp;fibonacci.py<br><br>def&nbsp;fibonacci(n:&nbsp;int)&nbsp;->&nbsp;int:<br>&nbsp;&nbsp;&nbsp;&nbsp;if&nbsp;n&nbsp;<=&nbsp;1:<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;return&nbsp;n<br>&nbsp;&nbsp;&nbsp;&nbsp;else:<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;return&nbsp;(fibonacci(n-1)&nbsp;+&nbsp;fibonacci(n-2))<br><br>def&nbsp;zero_pad(string:&nbsp;str)&nbsp;->&nbsp;str:<br>&nbsp;&nbsp;&nbsp;&nbsp;if&nbsp;len(string)&nbsp;>=&nbsp;10:<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;return&nbsp;string<br>&nbsp;&nbsp;&nbsp;&nbsp;else:<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;pad&nbsp;=&nbsp;'0'&nbsp;*&nbsp;(10&nbsp;-&nbsp;len(string))<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;return&nbsp;pad&nbsp;+&nbsp;string<br>,[\n# fibonacci.py\n\ndef fibonacci(n: int) -> int:\n if n <= 1:\n return n\n else:\n return (fibonacci(n-1) + fibonacci(n-2))\n\ndef zero_pad(string: str) -> str:\n if len(string) >= 10:\n return string\n else:\n pad = '0' * (10 - len(string))\n return pad + string\n],"```python\n# fibonacci.py\n\ndef fibonacci(n: int) -> int:\n if n <= 1:\n return n\n else:\n return (fibonacci(n-1) + fibonacci(n-2))\n\ndef zero_pad(string: str) -> str:\n if len(string) >= 10:\n return string\n else:\n pad = '0' * (10 - len(string))\n return pad + string\n```\nIn the above code, we have defined two functions: \n\n- `fibonacci(n: int) -> int`: returns the ""n""th Fibonacci number.\n- `zero_pad(string: str) -> str`: Takes a string as input and adds extra 0s to it to make it 10 characters long. If the string is already 10 or more characters, it is returned as-is. \n\nWe have saved these functions in the file `fibonacci.py`."


In [11]:
# A pandas style applies custom formatting to a DataFrame to make it easier to read and
# understand. It does not offer the full range of functionality that a DataFrame provides.
parser.styled_log

,id,created,model,usage,all_code
0,chatcmpl-73yLvjyfoIYg0n9OGyQibvV7HyHNx,1681181335.000000,gpt-3.5-turbo-0301,66.000000,# fibonacci.pydef fibonacci(n: int) -> int: if n <= 1: return n else: return (fibonacci(n-1) + fibonacci(n-2))def zero_pad(string: str) -> str: if len(string) >= 10: return string else: pad = '0' * (10 - len(string)) return pad + string


## And we make another exchange

In [13]:
message_list = f"""
system: {sys_prompt_string}
user: Write a python function that returns numbers 1-10 in a list. 
"""

message_parser = MessageParser(message_list)
messages = message_parser.parse_messages()
response = openai.ChatCompletion.create(model='gpt-3.5-turbo', temperature=1.3, max_tokens=300, messages = messages)
response_parser = ResponseParser(response=response)

logs.append(response_parser)

## And print the styled logs with the code from two separate prompt exchanges

In [14]:
logs.styled_log

,created,model,usage,all_code
id,,,,
nan,1681181335.000000,gpt-3.5-turbo-0301,66.000000,# fibonacci.pydef fibonacci(n: int) -> int: if n <= 1: return n else: return (fibonacci(n-1) + fibonacci(n-2))def zero_pad(string: str) -> str: if len(string) >= 10: return string else: pad = '0' * (10 - len(string)) return pad + string
chatcmpl-73yM40Sd3yhd8b1QVc9z2yxcfEwFv,1681181344.000000,gpt-3.5-turbo-0301,60.000000,"def get_numbers(): return [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]from myfunctions import get_numbersprint(get_numbers()) # Output# [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]print(numbers)# Output# [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]"
